# Clasificación de Cobertura del suelo

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pandas as pd
import rasterio
from sklearn.model_selection import train_test_split
from rasterio.plot import show_hist, show
import fiona
from rasterio.tools.mask import mask
from pandas.util.testing import assert_frame_equal
from rasterio import Affine
import geopandas as gpd

Cargando imágen principal y módelo de elevación digital


In [13]:
base_raster = rasterio.open('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/dos-reflectance.tif')
base_shapefile = '/Users/oscar/Code/Maestria/perc_remota/taller_2/datos_de_salida/Reflectancias/train/socorro.shp'

with fiona.open(base_shapefile, "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile] 
out_image, out_transform = mask(base_raster, geoms, crop=True)

out_image, out_transform = mask(base_raster, geoms, crop=True)
out_meta = base_raster.meta.copy()
out_meta.update({"driver": "GTiff",
  "height": out_image.shape[1],
  "width": out_image.shape[2],
  "transform": out_transform})
no_data=base_raster.nodata
data = out_image.data[0]
row, col = np.where(data >= no_data)
dataset = gpd.GeoDataFrame({'col':col,'row':row})


           col   row
0         3431     0
1         3430     1
2         3431     1
3         3432     1
4         3428     2
5         3429     2
6         3430     2
7         3431     2
8         3432     2
9         3427     3
10        3428     3
11        3429     3
12        3430     3
13        3431     3
14        3432     3
15        3433     3
16        3425     4
17        3426     4
18        3427     4
19        3428     4
20        3429     4
21        3430     4
22        3431     4
23        3432     4
24        3433     4
25        3434     4
26        3424     5
27        3425     5
28        3426     5
29        3427     5
...        ...   ...
14413296  4098  4395
14413297  4099  4395
14413298  4100  4395
14413299  4101  4395
14413300  4102  4395
14413301  4103  4395
14413302  4104  4395
14413303  4105  4395
14413304  4106  4395
14413305  4107  4395
14413306  4108  4395
14413307  4109  4395
14413308  4110  4395
14413309  4111  4395
14413310  4112  4395
14413311  411

Funciones necesarias para leer los polígonos e imágen ajustada al formato necesario para ser procesada por el algoritmo de clasificación.

In [168]:
def read_train_data(shape, class_id):
  print(base_raster)
  raster  = base_raster
  
  with fiona.open(shape, "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]  
  out_image, out_transform = mask(raster, geoms, crop=True)
  out_meta = raster.meta.copy()
  out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform})

  no_data=raster.nodata
  out_image2, out_transform2 = mask(dem_raster, geoms, crop=True)
  rows = out_image2[0].shape[0] if (out_image2[0].shape[0] < out_image[0].shape[0]) else out_image[0].shape[0]
  cols = out_image2[0].shape[1] if (out_image2[0].shape[1] < out_image[0].shape[1]) else out_image[0].shape[1]
  
  out_image=out_image[:,:rows,:cols]
  out_image2=out_image2[:,:rows,:cols]
    
  data = out_image.data[0]
  row, col = np.where(data >= no_data)
  d = gpd.GeoDataFrame({'col':col,'row':row})
  d['red']  = out_image.data[0].flatten()
  d['green']  = out_image.data[1].flatten()
  d['blue']  = out_image.data[2].flatten()
  d['nir']  = out_image.data[3].flatten()
  d['ndvi'] = d.apply(lambda row: ((row.nir - row.red)/(row.nir + row.red)), axis=1)
  d['elevation']  = out_image2.data[0].flatten()
  d['class']  = int(class_id)
  d = d[d['red'] != no_data]
  d['ndvi'] =  d['ndvi'] * 100
  filename = '/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/%s.csv' % (class_id)
  d.to_csv(filename, sep=',')
  return d
    

Leyendo polígonos de entrenamiento.


In [187]:
agua_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/agua.shp', 1)
arbustal_abierto_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/arbustal-abierto.shp', 2)
cafe_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/cafe.shp', 3)
mosaico1_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/mosaico-cultivos-pasto-espacios-naturales.shp', 4)
mosaico2_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/mosaico-pastos-cultivos.shp', 5)
urbano_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/urban.shp', 6)
pastos_limpios_data = read_train_data('/Users/oscar/Code/Maestria/perc_remota/taller_3/images/train/pastos-limpios.shp', 7)


<open RasterReader name='/Users/oscar/Code/Maestria/perc_remota/taller_2/datos_de_salida/Reflectancias/toasocorro.tif' mode='r'>
